In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True,projection=None,distinct=None):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    if projection == None:
        if distinct == None:
            cursor = db[collection].find(query)
        else:
            cursor = db[collection].find(query).distinct(distinct)
        
    elif projection != None and distinct == None:
        cursor = db[collection].find(query, projection)
        
    elif projection != None and distinct != None:
        cursor = db[collection].find(query, projection).distinct(distinct)
        
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        try:
            del df['_id']
        except:
            pass

    return df



In [2]:
dfPrices=read_mongo(db="meals",collection="prices",username="website",password="Don'tLogin",distinct="n"\
                    ,projection={"n":True},no_id=True)

In [3]:
dfRecipes=read_mongo(db="meals",collection="recipes",projection={"_id":True,"ingrd.n":True},username="website",password="Don'tLogin",no_id=False)

In [4]:
#dfPrices.to_csv('data.csv',index=False,encoding="utf-8-sig")

In [5]:
from ArabicIngredientsStemmer import ArabicIngredientsStemmer

In [6]:
Stemmer = ArabicIngredientsStemmer(dfPrices[0])

NameError: name 'ItemCorpus' is not defined

In [ ]:
ingrdsCorpus=Stemmer.extractNamesIngredients(dfRecipes[:5])

In [ ]:
results = Stemmer.machIngredItmPrice(ingrdsCorpus)

In [ ]:
results